In [ ]:
import os
import numpy as np
import cupy as cp
from scipy.io import mmread
import scipy.sparse as sp
import cupyx.scipy.sparse as csp
import matplotlib.pyplot as plt
import package.myUtil as myUtil
from PIL import Image
from IPython.display import display

In [ ]:
def gradient_operator(n):
    # 1Dの差分行列を作成
    e = np.ones(n)
    D1d = sp.diags([-e, e], [0, 1], shape=(n - 1, n))
    
    # 単位行列
    I_n = sp.eye(n)
    
    # Kronecker積を用いて2Dの勾配演算子を作成
    Dx = sp.kron(I_n, D1d)  # 水平方向の差分
    Dy = sp.kron(D1d, I_n)  # 垂直方向の差分
    
    return Dx.tocsr(), Dy.tocsr()

def divergence_operator(Dx, Dy):
    # 勾配演算子の随伴（負の発散演算子）
    DxT = -Dx.transpose()
    DyT = -Dy.transpose()
    return DxT.tocsr(), DyT.tocsr()

def proj_linf(y, tau):
    # 無限ノルムボールへの射影
    return np.clip(y, -tau, tau)

def proj_unit_interval(f):
    # [0,1]への射影
    return np.clip(f, 0, 1)

def primal_dual_solver(g, H, tau_reg, max_iter=500, tol=1e-4):
    n = int(np.sqrt(H.shape[1]))
    f = np.zeros(n * n)
    f_prev = f.copy()
    
    # 勾配と発散演算子
    Dx, Dy = gradient_operator(n)
    DxT, DyT = divergence_operator(Dx, Dy)
    K_norm = np.sqrt(8)
    tau = sigma = 0.99 / K_norm**2
    theta = 1.0

    # 双対変数の初期化
    yx = np.zeros(Dx.shape[0])
    yy = np.zeros(Dy.shape[0])
    
    for k in range(max_iter):
        # 外挿
        f_bar = f + theta * (f - f_prev)
    
        # φの勾配計算
        Hf = H.dot(f)
        grad_phi = -2 * H.transpose().dot(g - Hf)
    
        # K * f_barの計算（勾配）
        K_f_bar_x = Dx.dot(f_bar)
        K_f_bar_y = Dy.dot(f_bar)
    
        # 双対更新と射影
        yx = proj_linf(yx + sigma * K_f_bar_x, tau_reg)
        yy = proj_linf(yy + sigma * K_f_bar_y, tau_reg)
    
        # 発散の計算 K^T * y
        KTy = DxT.dot(yx) + DyT.dot(yy)
    
        # 主双対更新と射影
        f_prev = f.copy()
        f = proj_unit_interval(f - tau * (grad_phi + KTy))
    
        # 収束判定
        norm_diff = np.linalg.norm(f - f_prev)
        norm_f = np.linalg.norm(f_prev)
        if norm_f == 0:
            norm_f = 1.0  # ゼロ除算を避ける
        if norm_diff / norm_f < tol:
            print(f'{k+1}回の反復で収束しました。')
            break

    return f

In [ ]:
DATA_PATH = '../data'
OBJ_NAME = "Cameraman"
H_SETTING = "gf"
# H_SETTING = "int_p-5_lmd-100_to-True"
# H_SETTING = "p-5_lmd-100_to-False"
CAP_DATE = "241114"
EXP_DATE = "241202"
# 画像サイズ
n = 128
m = 255

In [ ]:
# Ground Truth
f_true = Image.open(f"{DATA_PATH}/sample_image{n}/{OBJ_NAME}.png").convert("L")

# システム行列 H
loaded = cp.load(f"{DATA_PATH}/{EXP_DATE}/systemMatrix/H_matrix_{H_SETTING}.npz")
H = csp.csr_matrix(
    (cp.array(loaded["data"]), cp.array(loaded["indices"]), cp.array(loaded["indptr"])), 
    shape=tuple(loaded["shape"])
)
print(H.shape)

# 観測画像 g
captured = Image.open(f"{DATA_PATH}/capture_{CAP_DATE}/{OBJ_NAME}.png").convert("L")
black = myUtil.calculate_bias(m**2, DATA_PATH, CAP_DATE)
g = captured.ravel() - black

In [ ]:
# 正則化パラメータ
tau_reg = 0.1

# 最適化問題を解く
f_reconstructed = primal_dual_solver(g, H, tau_reg)

# 結果の表示
f_reconstructed_image = f_reconstructed.reshape((n, n))
f_true_image = f_true.reshape((n, n))

plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.title("真の画像")
plt.imshow(f_true_image, cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.title("観測画像")
plt.imshow(g.reshape((n, n)), cmap="gray")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.title("再構成画像")
plt.imshow(f_reconstructed_image, cmap="gray")
plt.axis("off")

plt.show()

In [ ]:
f_image = Image.fromarray((f_reconstructed_image * 255).astype(np.uint8), mode="L")
display(f_image)

if not os.path.exists(f"{DATA_PATH}/{EXP_DATE}/reconst"):
    os.makedirs(f"{DATA_PATH}/{EXP_DATE}/reconst")
SAVE_PATH = f"{DATA_PATH}/{EXP_DATE}/reconst/{OBJ_NAME}_{H_SETTING}_primal_t-{tau_reg}.png"
f_image.save(SAVE_PATH, format="PNG")
print(SAVE_PATH)